In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract
from glob import glob
import spacy 
import re
import string 

In [2]:
def cleanText(txt):
    whitespace = string.whitespace
    punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~"
    tableWhitespace = str.maketrans('','',whitespace)
    tablePunctuation = str.maketrans('','',punctuation)
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [3]:
#load image 
#extract data usiong pytesseract
#conver data into content
#get predictions from NER model

In [4]:
#load model NER
model_ner=spacy.load('C:/Users/Shubham/clg project/1_bisscardNER/Version_2/output/model-best/')

In [5]:
#load image 
image = cv2.imread('C:/Users/Shubham/clg project/1_bisscardNER/Version_2/Images/6.jpeg')
tessData = pytesseract.image_to_data(image)
tesslist = list(map(lambda x:x.split('\t'), tessData.split('\n')))
df = pd.DataFrame(tesslist[1:],columns=tesslist[0])
df.dropna(inplace=True) #drop missing values
df['text'] = df['text'].apply(cleanText)

df_clean = df.query('text != "" ')
content = " ".join([w for w in df_clean['text']])
print(content)

#get predictions
doc = model_ner(content)

cell 8099948528 g2 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo


In [6]:
from spacy import displacy

In [7]:
displacy.serve(doc,style='ent')

C:\Users\Shubham\clg project\docuscan\lib\site-packages\spacy\displacy\__init__.py:108: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [8]:
#tagging

In [9]:
docjson = doc.to_json()
docjson.keys()

dict_keys(['text', 'ents', 'tokens'])

In [10]:
doc_text = docjson['text']
doc_text

'cell 8099948528 g2 8466045457 email lictsrikant@gmail.com life insurance corporation of india seosrika ntht@gmail “com thathineni srikanth insurance advisor agent code no. 0316164y life insurance corporation of india br. off. lic office, trimulgherry, sec’bad - 500 016. add. borabanda, hyderabad - 500 018. lictsrikant8099948528.blogspot.in, interviewsinhyderabad.blogspot.in facebook.com/lictsrikant8099948528, facebook.com/thathineni.srikanth.9 promote your business online pybo'

In [11]:
datafram_tokens = pd.DataFrame(docjson['tokens'])
datafram_tokens['tokens'] = datafram_tokens[['start','end']].apply(lambda x:doc_text[x[0]:x[1]],axis = 1)

datafram_tokens.head(10)

,id,start,end,tokens
0,0,0,4,cell
1,1,5,15,8099948528
2,2,16,18,g2
3,3,19,29,8466045457
4,4,30,35,email
5,5,36,57,lictsrikant@gmail.com
6,6,58,62,life
7,7,63,72,insurance
8,8,73,84,corporation
9,9,85,87,of


In [12]:
doc_text[73:84]

'corporation'

In [13]:
right_table = pd.DataFrame(docjson['ents'])[['start','label']]
datafram_tokens = pd.merge(datafram_tokens,right_table,how='left',on='start')

In [14]:
datafram_tokens.fillna('O',inplace=True)
datafram_tokens.head(10)

,id,start,end,tokens,label
0,0,0,4,cell,O
1,1,5,15,8099948528,B-PHONE
2,2,16,18,g2,I-PHONE
3,3,19,29,8466045457,B-PHONE
4,4,30,35,email,O
5,5,36,57,lictsrikant@gmail.com,B-EMAIL
6,6,58,62,life,B-ORG
7,7,63,72,insurance,I-ORG
8,8,73,84,corporation,O
9,9,85,87,of,O


In [15]:
#joining lable to df_clean dataframe
df_clean['end'] = df_clean['text'].apply (lambda x: len(x)+1).cumsum()- 1
df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)

C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\3036190664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['end'] = df_clean['text'].apply (lambda x: len(x)+1).cumsum()- 1
C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\3036190664.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['start'] = df_clean[['text','end']].apply(lambda x: x[1] - len(x[0]),axis=1)


In [16]:
#inner join with start 
dataframe_info = pd.merge(df_clean,datafram_tokens[['start','tokens','label']],how='inner',on='start')

In [17]:
dataframe_info.tail(10)

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label
41,5,1,6,3,1,6,495,474,53,22,95.485550,018.,307,303,018,I-ADD
42,5,1,6,3,2,1,46,506,430,27,83.094940,"lictsrikant8099948528.blogspot.in,",342,308,lictsrikant8099948528.blogspot.in,O
43,5,1,6,3,2,2,488,506,429,27,88.122208,interviewsinhyderabad.blogspot.in,376,343,interviewsinhyderabad.blogspot.in,O
44,5,1,6,3,3,1,44,539,472,25,88.884369,"facebook.com/lictsrikant8099948528,",412,377,facebook.com/lictsrikant8099948528,O
45,5,1,6,3,3,2,526,539,443,21,91.430847,facebook.com/thathineni.srikanth.9,447,413,facebook.com/thathineni.srikanth.9,O
46,5,1,6,3,4,1,46,571,106,21,96.386932,promote,455,448,promote,O
47,5,1,6,3,4,2,161,576,56,22,96.813004,your,460,456,your,O
48,5,1,6,3,4,3,226,571,111,21,96.439957,business,469,461,business,O
49,5,1,6,3,4,4,347,571,74,21,93.266945,online,476,470,online,O
50,5,1,6,3,4,5,432,571,97,27,92.782974,pybo,481,477,pybo,O


In [18]:
 #bounding box

In [19]:
bb_df = dataframe_info.query(" label !='O' ")
img = image.copy()

for x,y,w,h,label in bb_df[['left','top','width','height','label']].values:
    x = int(x)
    y = int(y)
    w = int(w)
    h = int(h)
    
    cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.putText(img,str(label),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
    
cv2.imshow('predictions',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label
1,5,1,3,1,1,3,822,53,203,28,96.606537,8099948528,15,5,8099948528,B-PHONE
2,5,1,3,2,1,1,55,55,85,89,46.902012,g2,18,16,g2,I-PHONE
3,5,1,3,2,1,2,822,95,203,28,96.438461,8466045457,29,19,8466045457,B-PHONE
5,5,1,3,2,2,3,709,136,316,31,88.662399,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,B-EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.542465,life,62,58,life,B-ORG
7,5,1,3,3,1,2,85,151,92,42,92.259232,insurance,72,63,insurance,I-ORG
11,5,1,3,3,1,6,667,166,120,37,92.730202,seosrika,102,94,seosrika,B-EMAIL
12,5,1,3,3,1,7,791,165,162,44,91.439003,ntht@gmail,113,103,ntht@gmail,B-PHONE
13,5,1,3,3,1,8,958,180,67,19,32.106407,“com,118,114,“,I-EMAIL
15,5,1,4,1,1,2,557,228,198,31,96.660568,srikanth,138,130,srikanth,B-NAME


In [21]:
bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])
bb_df.head()

C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\2854881843.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['label'] = bb_df['label'].apply(lambda x: x[2:])


,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label
1,5,1,3,1,1,3,822,53,203,28,96.606537,8099948528,15,5,8099948528,PHONE
2,5,1,3,2,1,1,55,55,85,89,46.902012,g2,18,16,g2,PHONE
3,5,1,3,2,1,2,822,95,203,28,96.438461,8466045457,29,19,8466045457,PHONE
5,5,1,3,2,2,3,709,136,316,31,88.662399,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL
6,5,1,3,3,1,1,46,170,33,14,96.542465,life,62,58,life,ORG


In [22]:
#group the labels 
class groupgen():
    def __init__(self):
        self.id = 0
        self.text = ''
        
    def getgroup(self,text):
        if self.text == text:
            return self.id
        else:
            self.id +=1
            self.text = text
            return self.id
        
grp_gen = groupgen()

In [23]:
bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\293095595.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['group'] = bb_df['label'].apply(grp_gen.getgroup)


In [24]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label,group
1,5,1,3,1,1,3,822,53,203,28,96.606537,8099948528,15,5,8099948528,PHONE,1
2,5,1,3,2,1,1,55,55,85,89,46.902012,g2,18,16,g2,PHONE,1
3,5,1,3,2,1,2,822,95,203,28,96.438461,8466045457,29,19,8466045457,PHONE,1
5,5,1,3,2,2,3,709,136,316,31,88.662399,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2
6,5,1,3,3,1,1,46,170,33,14,96.542465,life,62,58,life,ORG,3
7,5,1,3,3,1,2,85,151,92,42,92.259232,insurance,72,63,insurance,ORG,3
11,5,1,3,3,1,6,667,166,120,37,92.730202,seosrika,102,94,seosrika,EMAIL,4
12,5,1,3,3,1,7,791,165,162,44,91.439003,ntht@gmail,113,103,ntht@gmail,PHONE,5
13,5,1,3,3,1,8,958,180,67,19,32.106407,“com,118,114,“,EMAIL,6
15,5,1,4,1,1,2,557,228,198,31,96.660568,srikanth,138,130,srikanth,NAME,7


In [25]:
#create right and bottom of bonding box
bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
bb_df['right'] = bb_df['left'] + bb_df['width']
bb_df['bottom'] = bb_df['top'] + bb_df['height']

C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\160152939.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df[['left','top','width','height']] = bb_df[['left','top','width','height']].astype(int)
C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\160152939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bb_df['right'] = bb_df['left'] + bb_df['width']
C:\Users\Shubham\AppData\Local\Temp\ipykernel_15404\160152939.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [26]:
bb_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,end,start,tokens,label,group,right,bottom
1,5,1,3,1,1,3,822,53,203,28,96.606537,8099948528,15,5,8099948528,PHONE,1,1025,81
2,5,1,3,2,1,1,55,55,85,89,46.902012,g2,18,16,g2,PHONE,1,140,144
3,5,1,3,2,1,2,822,95,203,28,96.438461,8466045457,29,19,8466045457,PHONE,1,1025,123
5,5,1,3,2,2,3,709,136,316,31,88.662399,lictsrikant@gmail.com,57,36,lictsrikant@gmail.com,EMAIL,2,1025,167
6,5,1,3,3,1,1,46,170,33,14,96.542465,life,62,58,life,ORG,3,79,184
7,5,1,3,3,1,2,85,151,92,42,92.259232,insurance,72,63,insurance,ORG,3,177,193
11,5,1,3,3,1,6,667,166,120,37,92.730202,seosrika,102,94,seosrika,EMAIL,4,787,203
12,5,1,3,3,1,7,791,165,162,44,91.439003,ntht@gmail,113,103,ntht@gmail,PHONE,5,953,209
13,5,1,3,3,1,8,958,180,67,19,32.106407,“com,118,114,“,EMAIL,6,1025,199
15,5,1,4,1,1,2,557,228,198,31,96.660568,srikanth,138,130,srikanth,NAME,7,755,259


In [27]:
#tagging : groupby group
col_group = ['left','top','right','bottom','label','tokens','group']
group_tag_img = bb_df[col_group].groupby(by='group')

In [28]:
img_tagging = group_tag_img.agg({
    
    'left':min,
    'right':max,
    'top':min,
    'bottom':max,
    'label':np.unique,
    'tokens':lambda x: " ".join(x)
})

In [29]:
img_tagging

,left,right,top,bottom,label,tokens
group,,,,,,
1,55,1025,53,144,[PHONE],8099948528 g2 8466045457
2,709,1025,136,167,[EMAIL],lictsrikant@gmail.com
3,46,177,151,193,[ORG],life insurance
4,667,787,166,203,[EMAIL],seosrika
5,791,953,165,209,[PHONE],ntht@gmail
6,958,1025,180,199,[EMAIL],“
7,557,755,228,259,[NAME],srikanth
8,400,546,271,296,[DES],insurance
9,47,130,396,426,[NAME],life


In [30]:
img_bb = image.copy()
for l,r,t,b,label,tokens in img_tagging.values:
    cv2.rectangle(img_bb,(l,t),(r,b),(0,255,0),2)
    
    cv2.putText(img_bb,str(label),(l,t),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
    
cv2.imshow('Bounding Box BusinessCard',img_bb)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [34]:
#parser (mixing tagging B with i )
def parser(text,label):
    if label == 'PHONE':
        text = text.lower()
        text = re.sub(r'\D','',text)
        
    elif label == 'EMAIL':
        text = text.lower()
        allow_special_char = '@)_.\-'
        text = re.sub(r'[^A-za-z0-9{}]' .format(allow_special_char),'',text)
        
    elif label == 'WEB':
        text = text.lower()
        allow_special_char = ':/.%\-'
        text = re.sub(r'[^A-za-z0-9{}]' .format(allow_special_char),'',text)
        
    elif label in ('NAME', 'DES'):
        text = text.lower()
        text = re.sub(r'[^a-z ]','',text)
        text = text.title()
                   
    elif label == 'ORG':
        text = text.lower()
        text = re.sub(r'[^a-z0-9]','',text)
        text = text.title()
        
    elif label == 'ADD':
        text = text.lower()
        text = re.sub(r'[^a-z0-9]','',text)
        text = text.title()
        
    return text

In [35]:
parser('shubham160#$%^&@GMAIL>COM','EMAIL')

'shubham160^@gmailcom'

In [43]:
#entities
info_array = dataframe_info[['tokens','label']].values
entities = dict(NAME=[],ORG=[],DES=[],ADD=[],PHONE=[],EMAIL=[],WEB=[])
previous = 'O'

for tokens, label in info_array:
    #print(tokens,label)
    bio_tag = label[:1]
    label_tag = label[2:]
    
    #step 1 parse the tokens
    text = parser(tokens,label_tag)
    
    if bio_tag in ('B','I'):
        
        if previous != label_tag:
            entities[label_tag].append(text)
            
        else:
            if bio_tag == "B":
                entities[label_tag].append(text)
    
            else: 
                if label_tag in ("NAME",'ORG','DES','ADD'):
                    entities[label_tag][-1] = entities[label_tag][-1] + " " + text
                    
                else:
                    entities[label_tag][-1] = entities[label_tag][-1] + text
    
    previous = label_tag

In [44]:
entities

{'NAME': ['Srikanth', 'Life'],
 'ORG': ['Life Insurance', 'Of', 'Off', 'Add'],
 'DES': ['Insurance', 'Insurance'],
 'ADD': ['016', ' 500 018'],
 'PHONE': ['80999485282', '8466045457', ''],
 'EMAIL': ['lictsrikant@gmail.com', 'seosrika', ''],
 'WEB': []}

In [45]:
#prediction function
